In [21]:
from z3 import *
import networkx as nx
import matplotlib.pyplot as plt
from tabulate import tabulate
import numpy as np
import pprint
from graphviz import Digraph


# ROB scheduling adversaries


I also want to see adversarial imputs to the complete OoO vs the dataflow...

Why going from 6 to 8 or bigger is a big deal Can you comment on that?

Assuming instructions can be executed in any FU, but only one at a time

Assuming I can retire as many instructions per cycle as I want

## Parameters

In [22]:
NUM_INST = 5
MEM_TIME = 200
MAX_EXEC_TIME = 5
MIN_EXEC_TIME = 1

ROB_SIZE = 4
# RS_SIZE = 3 # Why do I have this lmao

ISSUE_WIDTH = 2
DISPATHCH_WIDTH = 2

NUM_FUNCTIONAL_UNITS = 10
NUM_IT = 10 * 2

In [23]:
example_dependency = [
    [0, 0, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 1, 1, 0]
]

example_exec_time = [1 for i in range(NUM_INST)]

## Variables

In [24]:

def data_dependency(producer, consumer):
    # return example_dependency[consumer][producer] == 1
    return Bool(f"data_dependency_{producer}_{consumer}")

def exec_time(inst):
    # return example_exec_time[inst]
    return Int(f"exec_time_{inst}")

def event_time(i, prefix="heuristic"):
    return Real(f"{prefix}_event_time_{i}")

def inst_issued(inst, i, prefix="heuristic"):
    return Bool(f"{prefix}_inst_issued_{inst}_{i}")

def inst_dispatched(inst, i, prefix="heuristic"):
    return Bool(f"{prefix}_inst_dispatched_{inst}_{i}")

def inst_completed(inst, i, prefix="heuristic"):
    return Bool(f"{prefix}_inst_completed_{inst}_{i}")

def inst_retired(inst, i, prefix="heuristic"):
    return Bool(f"{prefix}_inst_retired_{inst}_{i}")

def inst_end(inst, i, prefix="heuristic"):
    return Real(f"{prefix}_inst_end_{inst}_{i}")

In [25]:
s = Solver()

## Constraints

### Dependency matrix is lower triangle only

In [26]:
for producer in range(NUM_INST):
    for consumer in range(NUM_INST):
        if consumer <= producer:
            s.add(Not(data_dependency(producer, consumer)))

for inst in range(NUM_INST):
    s.add(Or(And(exec_time(inst) > MIN_EXEC_TIME), exec_time(inst) <= MAX_EXEC_TIME), exec_time(inst) == MEM_TIME)

### Can depend only on two other instructions

In [27]:
for consumer in range(NUM_INST):
    s.add(AtMost(*[data_dependency(producer, consumer) for producer in range(NUM_INST)], 2))

### Heuristic

In [28]:
constrains = []
for inst in range(NUM_INST):
    constrains.append(inst_issued(inst, 0) == False)
    constrains.append(inst_dispatched(inst, 0) == False)
    constrains.append(inst_completed(inst, 0) == False)
    constrains.append(inst_retired(inst, 0) == False)
    constrains.append(inst_end(inst, 0) == 0)

constrains.append(event_time(0) == 0)



eval_just_finished = []
eval_ready = []
eval_can_retire = []
eval_youngest_retired = []
eval_youngest_issued = []
eval_issue = []
eval_free_fu = []
eval_dispatched = []
eval_num_dispatched = []
eval_min = []

In [29]:
for i in range(NUM_IT):
    just_finished = []
    for inst in range(NUM_INST):
        just_finished.append(And(inst_dispatched(inst, i), inst_end(inst, i) == event_time(i)))

    ready = []
    for consumer in range(NUM_INST):
        ready.append(And(*[Or(Not(data_dependency(producer, consumer)), Or(just_finished[producer], inst_completed(producer, i), inst_retired(producer, i))) for producer in range(NUM_INST)], inst_issued(consumer, i)))

    can_retire = []
    for inst in range(NUM_INST):
        can_retire.append(And(*[Or(inst_retired(old, i), just_finished[old], inst_completed(old, i)) for old in range(inst)], Or(just_finished[inst], inst_completed(inst, i))))

    youngest_retired = - 1
    for inst in range(NUM_INST):
        youngest_retired = If(inst_retired(inst, i), inst, youngest_retired) # NOTE: Not considering can_retire, this means I can't pop and push to ROB at the same time if ROB was full

    youngest_issued = - 1
    for inst in range(NUM_INST): # NOTE: Check is 0 to NUM or NUM to 0...
        youngest_issued = If(And(inst_issued(inst, i), Not(inst_retired(inst, i))), inst, youngest_issued)
    
    issue = []
    for inst in range(NUM_INST):
        issue.append(And(Not(inst_issued(inst, i)), inst - youngest_retired <= ROB_SIZE, inst - youngest_issued <= ISSUE_WIDTH))

    # 1 1 0 0 0 0 0 0 0 
    # 1 1 

    free_fu = NUM_FUNCTIONAL_UNITS
    for inst in range(NUM_INST):
        free_fu = If(And(inst_dispatched(inst, i), Not(just_finished[inst])), free_fu - 1, free_fu)
    
    dispatched = []
    num_dispatched = [RealVal(0)]
    for inst in range(NUM_INST):
        dispatched.append(And(ready[inst], Not(inst_dispatched(inst, i)), inst_issued(inst, i) , num_dispatched[inst] < DISPATHCH_WIDTH, free_fu - num_dispatched[inst] <= NUM_FUNCTIONAL_UNITS))
        num_dispatched.append(If(dispatched[inst], num_dispatched[inst] + 1, num_dispatched[inst]))
        


    for inst in range(NUM_INST):
        constrains.append(If(Or(inst_issued(inst, i), issue[inst]), inst_issued(inst, i + 1), Not(inst_issued(inst, i + 1))))
        constrains.append(If(Or(inst_dispatched(inst, i), dispatched[inst]), inst_dispatched(inst, i + 1), Not(inst_dispatched(inst, i + 1))))
        constrains.append(If(dispatched[inst], inst_end(inst, i + 1) == event_time(i) + exec_time(inst), inst_end(inst, i) == inst_end(inst, i + 1)))
        constrains.append(If(Or(inst_retired(inst, i), can_retire[inst]), inst_retired(inst, i + 1), Not(inst_retired(inst, i + 1))))
        constrains.append(If(Or(inst_completed(inst, i), just_finished[inst]), inst_completed(inst, i + 1), Not(inst_completed(inst, i + 1))))
    
    

    min_fu = 100000
    for inst in range(NUM_INST):
        # in_some_fu = Or(*[inst_assigned_fu(inst, fu, i + 1) for fu in range(NUM_FUNCTIONAL_UNITS)])
        min_fu = If(And(inst_end(inst, i + 1) < min_fu, inst_dispatched(inst, i + 1),  inst_end(inst, i + 1) > event_time(i)), inst_end(inst, i + 1), min_fu)

    # print(min_fu)
    # constrains.append(min_fu > event_time(i))
    constrains.append(If(min_fu == 100000, event_time(i + 1) == event_time(i), event_time(i + 1) == min_fu))
    
    

    eval_just_finished.append(just_finished)
    eval_ready.append(ready)
    eval_can_retire.append(can_retire)
    eval_youngest_retired.append(youngest_retired)
    eval_youngest_issued.append(youngest_issued)
    eval_issue.append(issue)
    eval_free_fu.append(free_fu)
    eval_dispatched.append(dispatched)
    eval_num_dispatched.append(num_dispatched)
    eval_min.append(min_fu)

### Optimal

In [30]:
opt_constrains = []
for inst in range(NUM_INST):
    opt_constrains.append(inst_issued(inst, 0, "opt") == False)
    opt_constrains.append(inst_dispatched(inst, 0, "opt") == False)
    opt_constrains.append(inst_completed(inst, 0, "opt") == False)
    opt_constrains.append(inst_retired(inst, 0, "opt") == False)
    opt_constrains.append(inst_end(inst, 0, "opt") == 0)

opt_constrains.append(event_time(0, "opt") == 0)

opt_eval_just_finished = []
opt_eval_ready = []
opt_eval_can_retire = []
opt_eval_youngest_retired = []
opt_eval_youngest_issued = []
opt_eval_issue = []
opt_eval_free_fu = []
opt_eval_dispatched = []
opt_eval_num_dispatched = []
opt_eval_min = []

In [31]:
for i in range(NUM_IT):
    just_finished = []
    for inst in range(NUM_INST):
        just_finished.append(And(inst_dispatched(inst, i, prefix="opt"), inst_end(inst, i, prefix="opt") == event_time(i, prefix="opt")))

    ready = []
    for consumer in range(NUM_INST):
        ready.append(And(*[Or(Not(data_dependency(producer, consumer)), Or(just_finished[producer], inst_completed(producer, i, prefix="opt"), inst_retired(producer, i, prefix="opt"))) for producer in range(NUM_INST)], inst_issued(consumer, i, prefix="opt")))

    can_retire = []
    for inst in range(NUM_INST):
        can_retire.append(And(*[Or(inst_retired(old, i, prefix="opt"), just_finished[old], inst_completed(old, i, prefix="opt")) for old in range(inst)], Or(just_finished[inst], inst_completed(inst, i, prefix="opt"))))

    youngest_retired = - 1
    for inst in range(NUM_INST):
        youngest_retired = If(inst_retired(inst, i, prefix="opt"), inst, youngest_retired) # NOTE: Not considering can_retire, this means I can't pop and push to ROB at the same time if ROB was full

    youngest_issued = - 1
    for inst in range(NUM_INST): # NOTE: Check is 0 to NUM or NUM to 0...
        youngest_issued = If(And(inst_issued(inst, i, prefix="opt"), Not(inst_retired(inst, i, prefix="opt"))), inst, youngest_issued)
    
    issue = []
    for inst in range(NUM_INST):
        issue.append(And(Not(inst_issued(inst, i, prefix="opt")), inst - youngest_retired <= ROB_SIZE, inst - youngest_issued <= ISSUE_WIDTH))

    # 1 1 0 0 0 0 0 0 0 
    # 1 1 

    free_fu = NUM_FUNCTIONAL_UNITS
    for inst in range(NUM_INST):
        free_fu = If(And(inst_dispatched(inst, i, prefix="opt"), Not(just_finished[inst])), free_fu - 1, free_fu)
    
    dispatched = []
    for inst in range(NUM_INST):
        dispatched.append(And(ready[inst], Not(inst_dispatched(inst, i, prefix="opt")), inst_issued(inst, i, prefix="opt"), Bool(f"to_be_dispatched_{inst}_{i}")))
    
    num_dispatched = 0
    for inst in range(NUM_INST):
        num_dispatched = If(dispatched[inst], num_dispatched + 1, num_dispatched)
        
    opt_constrains.append(free_fu - num_dispatched <= NUM_FUNCTIONAL_UNITS)
    opt_constrains.append(num_dispatched <= DISPATHCH_WIDTH)


    for inst in range(NUM_INST):
        opt_constrains.append(If(Or(inst_issued(inst, i, prefix="opt"), issue[inst]), inst_issued(inst, i + 1, prefix="opt"), Not(inst_issued(inst, i + 1, prefix="opt"))))
        opt_constrains.append(If(Or(inst_dispatched(inst, i, prefix="opt"), dispatched[inst]), inst_dispatched(inst, i + 1, prefix="opt"), Not(inst_dispatched(inst, i + 1, prefix="opt"))))
        opt_constrains.append(If(dispatched[inst], inst_end(inst, i + 1, prefix="opt") == event_time(i, prefix="opt") + exec_time(inst), inst_end(inst, i, prefix="opt") == inst_end(inst, i + 1, prefix="opt")))
        opt_constrains.append(If(Or(inst_retired(inst, i, prefix="opt"), can_retire[inst]), inst_retired(inst, i + 1, prefix="opt"), Not(inst_retired(inst, i + 1, prefix="opt"))))
        opt_constrains.append(If(Or(inst_completed(inst, i, prefix="opt"), just_finished[inst]), inst_completed(inst, i + 1, prefix="opt"), Not(inst_completed(inst, i + 1, prefix="opt"))))
    
    

    min_fu = 100000
    for inst in range(NUM_INST):
        # in_some_fu = Or(*[inst_assigned_fu(prefix="opt", inst, fu, i + 1) for fu in range(NUM_FUNCTIONAL_UNITS)])
        min_fu = If(And(inst_end(inst, i + 1, prefix="opt") < min_fu, inst_dispatched(inst, i + 1, prefix="opt"),  inst_end(inst, i + 1, prefix="opt") > event_time(i, prefix="opt")), inst_end(inst, i + 1, prefix="opt"), min_fu)

    # print(min_fu)
    # opt_constrains.append(min_fu > event_time(i, prefix="opt"))
    opt_constrains.append(If(min_fu == 100000, event_time(i + 1, prefix="opt") == event_time(i, prefix="opt"), event_time(i + 1, prefix="opt") == min_fu))

    
    opt_eval_just_finished.append(just_finished)
    opt_eval_ready.append(ready)
    opt_eval_can_retire.append(can_retire)
    opt_eval_youngest_retired.append(youngest_retired)
    opt_eval_youngest_issued.append(youngest_issued)
    opt_eval_issue.append(issue)
    opt_eval_free_fu.append(free_fu)
    opt_eval_dispatched.append(dispatched)
    opt_eval_num_dispatched.append(num_dispatched)
    opt_eval_min.append(min_fu)
    
    

## Performance queries

In [32]:
queries = []
GAP = 1
queries.append(event_time(NUM_IT - 1, "opt") + GAP <= event_time(NUM_IT - 1))
queries.append(And(*[inst_retired(inst, NUM_IT - 1, "opt") for inst in range(NUM_INST)]))


In [33]:
s.add(*constrains)
s.add(*opt_constrains)
s.add(*queries)

## Results

In [34]:
check = s.check()
print(check)

sat


### Dependencies

In [35]:
dependency_matrix = []
if check == sat:
    model = s.model()
    for consumer in range(NUM_INST):
        aux = []
        for producer in range(NUM_INST):
            aux.append(1 if model[data_dependency(producer=producer, consumer=consumer)] else 0)
        dependency_matrix.append(aux)
    
    print(tabulate(dependency_matrix, tablefmt="grid"))

    timing = []
    for inst in range(NUM_INST):
        timing.append(model[exec_time(inst)])
    # print(timing)
    dot = Digraph()
    for i in range(NUM_INST):
        dot.node(f"{i}", f"{i}\n cycles {model[exec_time(i)]}")

    for consumer in range(NUM_INST):
        for producer in range(NUM_INST):
            if dependency_matrix[consumer][producer]:
                dot.edges([(f"{producer}", f"{consumer}")])

    dot.render(filename=f"graphs/NUM_INST_{NUM_INST}_MAX_EXEC_TIME_{MAX_EXEC_TIME}_MIN_EXEC_TIME_{MIN_EXEC_TIME}_MEM_TIME_{MEM_TIME}_ROB_SIZE_{ROB_SIZE}_ISSUE_WIDTH_{ISSUE_WIDTH}_DISPATCH_WIDTH_{DISPATHCH_WIDTH}_NUM_FUNCTIONAL_UNITS_{NUM_FUNCTIONAL_UNITS}_NUM_IT_{NUM_IT}_GAP_{GAP}")


+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 1 | 0 | 1 | 0 | 0 |
+---+---+---+---+---+
| 0 | 1 | 0 | 1 | 0 |
+---+---+---+---+---+


### Optimal scheduling

In [36]:
if check == sat:
    model = s.model()
    
    for i in range(NUM_IT):        
        print(f"Event: {i} Time: {model[event_time(i, "opt")]}")
        print(tabulate([[str(model.evaluate(opt_eval_youngest_retired[i], model_completion = True)), str(model.evaluate(opt_eval_youngest_issued[i], model_completion = True)), str(model.evaluate(opt_eval_free_fu[i], model_completion = True)), str(model.evaluate(opt_eval_num_dispatched[i], model_completion = True)), str(model.evaluate(opt_eval_min[i], model_completion = True))]], ['Youngest retired', 'Youngest_issued', 'Free FU', 'Dispatched', 'Min'], tablefmt="grid"))
        table = []
        for inst in range(NUM_INST): 
            entry = [inst, str(model[inst_issued(inst, i, "opt")]), str(model[inst_dispatched(inst, i, "opt")]), str(model[inst_completed(inst, i, "opt")]), str(model[inst_retired(inst, i, "opt")]), str(model[inst_end(inst, i, "opt")]),
                        str(model.evaluate(opt_eval_just_finished[i][inst], model_completion = True)),
                        str(model.evaluate(opt_eval_dispatched[i][inst], model_completion = True)),
                        
                        str(model.evaluate(opt_eval_ready[i][inst], model_completion = True)),
                        str(model.evaluate(opt_eval_can_retire[i][inst], model_completion = True)),
                        str(model.evaluate(opt_eval_issue[i][inst], model_completion = True))
                     ]
            
            table.append(entry) 
        print(tabulate(table, headers=['Instruction', 'Issued', 'Dispatched', 'Completed', 'Retired', 'End Time', 'Just finsiehd', 'To be dispatched', 'ready', 'can retire', 'to be issued'], tablefmt="grid"))

Event: 0 Time: 0
+--------------------+-------------------+-----------+--------------+-------+
|   Youngest retired |   Youngest_issued |   Free FU |   Dispatched |   Min |
+====================+===================+===========+==============+=======+
|                 -1 |                -1 |        10 |            0 |  1000 |
+--------------------+-------------------+-----------+--------------+-------+
+---------------+----------+--------------+-------------+-----------+------------+-----------------+--------------------+---------+--------------+----------------+
|   Instruction | Issued   | Dispatched   | Completed   | Retired   |   End Time | Just finsiehd   | To be dispatched   | ready   | can retire   | to be issued   |
+===============+==========+==============+=============+===========+============+=================+====================+=========+==============+================+
|             0 | False    | False        | False       | False     |          0 | False           | 

In [37]:
if check == sat:
    model = s.model()
    for i in range(NUM_IT):

        DATA = []
        print(f"Event {i} {model[event_time(i, "opt")]}")
        for inst in range(NUM_INST):
            rob_entry = ""
            if not model[inst_retired(inst, i, "opt")] and model[inst_issued(inst, i, "opt")]:
                rob_entry = str(inst)

            rs_entry = ""
            if not model[inst_dispatched(inst, i, "opt")] and model[inst_issued(inst, i, "opt")]:
                rs_entry = str(inst)
            
            fu_entry = ""
            end_time = ""
            if not model[inst_retired(inst, i, "opt")] and  model[inst_dispatched(inst, i, "opt")]:
                fu_entry = str(inst)
                end_time = str(model[inst_end(inst, i, "opt")])
            DATA.append([rob_entry, rs_entry, fu_entry, end_time])
        
        print(tabulate(DATA, ['ROB', 'RS', 'FU', 'Time'], tablefmt='grid'))



Event 0 0
+-------+------+------+--------+
| ROB   | RS   | FU   | Time   |
+=======+======+======+========+
|       |      |      |        |
+-------+------+------+--------+
|       |      |      |        |
+-------+------+------+--------+
|       |      |      |        |
+-------+------+------+--------+
|       |      |      |        |
+-------+------+------+--------+
|       |      |      |        |
+-------+------+------+--------+
Event 1 0
+-------+------+------+--------+
| ROB   | RS   | FU   | Time   |
+=======+======+======+========+
| 0     | 0    |      |        |
+-------+------+------+--------+
| 1     | 1    |      |        |
+-------+------+------+--------+
|       |      |      |        |
+-------+------+------+--------+
|       |      |      |        |
+-------+------+------+--------+
|       |      |      |        |
+-------+------+------+--------+
Event 2 0
+-------+------+------+--------+
| ROB   | RS   | FU   | Time   |
+=======+======+======+========+
| 0     | 0  

### Heuristic scheduling

In [38]:
if check == sat:
    model = s.model()
    
    for i in range(NUM_IT):
        
        print(f"Event: {i} Time: {model[event_time(i)]}")
        print(tabulate([[str(model.evaluate(eval_youngest_retired[i], model_completion = True)), str(model.evaluate(eval_youngest_issued[i], model_completion = True)), str(model.evaluate(eval_free_fu[i], model_completion = True)), str(model.evaluate(eval_min[i], model_completion = True))]], ['Youngest retired', 'Youngest_issued', 'Free FU', 'Min'], tablefmt="grid"))
       
        table = []
        for inst in range(NUM_INST): 
            entry = [inst, str(model[inst_issued(inst, i)]), str(model[inst_dispatched(inst, i)]), str(model[inst_completed(inst, i)]), str(model[inst_retired(inst, i)]), str(model[inst_end(inst, i)]),
                        str(model.evaluate(eval_just_finished[i][inst], model_completion = True)),
                        str(model.evaluate(eval_dispatched[i][inst], model_completion = True)),
                        str(model.evaluate(eval_num_dispatched[i][inst], model_completion = True)),
                        str(model.evaluate(eval_ready[i][inst], model_completion = True)),
                        str(model.evaluate(eval_can_retire[i][inst], model_completion = True)),
                        str(model.evaluate(eval_issue[i][inst], model_completion = True))
                     ]
            
            table.append(entry) 
        print(tabulate(table, headers=['Instruction', 'Issued', 'Dispatched', 'Completed', 'Retired', 'End Time', 'Just finsiehd', 'To be dispatched', 'Num dispatched', 'ready', 'can retire', 'to be issued'], tablefmt="grid"))

Event: 0 Time: 0
+--------------------+-------------------+-----------+-------+
|   Youngest retired |   Youngest_issued |   Free FU |   Min |
+====================+===================+===========+=======+
|                 -1 |                -1 |        10 |  1000 |
+--------------------+-------------------+-----------+-------+
+---------------+----------+--------------+-------------+-----------+------------+-----------------+--------------------+------------------+---------+--------------+----------------+
|   Instruction | Issued   | Dispatched   | Completed   | Retired   |   End Time | Just finsiehd   | To be dispatched   |   Num dispatched | ready   | can retire   | to be issued   |
+===============+==========+==============+=============+===========+============+=================+====================+==================+=========+==============+================+
|             0 | False    | False        | False       | False     |          0 | False           | False             

#### Deglossed into ROB, RS, INST and FUs

In [39]:

if check == sat:
    model = s.model()
    for i in range(NUM_IT):

        DATA = []
        print(f"Event {i} {model[event_time(i)]}")
        for inst in range(NUM_INST):
            rob_entry = ""
            if not model[inst_retired(inst, i)] and model[inst_issued(inst, i)]:
                rob_entry = str(inst)

            rs_entry = ""
            if not model[inst_dispatched(inst, i)] and model[inst_issued(inst, i)]:
                rs_entry = str(inst)
            
            fu_entry = ""
            end_time = ""
            if not model[inst_retired(inst, i)] and  model[inst_dispatched(inst, i)]:
                fu_entry = str(inst)
                end_time = str(model[inst_end(inst, i)])
            DATA.append([rob_entry, rs_entry, fu_entry, end_time])
        
        print(tabulate(DATA, ['ROB', 'RS', 'FU', 'Time'], tablefmt='grid'))



Event 0 0
+-------+------+------+--------+
| ROB   | RS   | FU   | Time   |
+=======+======+======+========+
|       |      |      |        |
+-------+------+------+--------+
|       |      |      |        |
+-------+------+------+--------+
|       |      |      |        |
+-------+------+------+--------+
|       |      |      |        |
+-------+------+------+--------+
|       |      |      |        |
+-------+------+------+--------+
Event 1 0
+-------+------+------+--------+
| ROB   | RS   | FU   | Time   |
+=======+======+======+========+
| 0     | 0    |      |        |
+-------+------+------+--------+
| 1     | 1    |      |        |
+-------+------+------+--------+
|       |      |      |        |
+-------+------+------+--------+
|       |      |      |        |
+-------+------+------+--------+
|       |      |      |        |
+-------+------+------+--------+
Event 2 200
+-------+------+------+--------+
| ROB   | RS   | FU   | Time   |
+=======+======+======+========+
| 0     |  